**Vanessa Navarro Coronado e Iván Sánchez Castellanos**

https://www.kaggle.com/kanncaa1/roc-curve-with-k-fold-cv

# Práctica 2 - Clasificación supervisada en scikit-learn --> VOICE DATASET

En esta práctica vamos a estudiar el dataset Voice. Probaremos los clasificadores de Árbol de Decisión y KNN, y analizaremos cuál es mejor en cada caso. Además, trataremos de descubrir las configuraciones óptimas de dichos algoritmos, y realizaremos un breve estudio sobre ellos.

Además, en la parte extra utilizaremos un Naive Bayes, usando el mismo procedimiento que seguimos para los clasificadores anteriormente citados.

Para comenzar, como siempre, lo primero que hacemos es importar los paquetes necesarios, e inicializar nuestra semilla para nuestros experimentos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(6342)

### Cargado de los datos

A continuación, cargamos los datos del dataset Voice. Este será separado en dos conjuntos, uno con los atributos y otro con la clase del dataset. Esto es necesario para que la libreria scikit learn pueda ser utilizada.

In [4]:
# Update the file path to fit your system
dfVoice = pd.read_csv("../data/voice.csv", dtype={ "label": 'category'})
dfAttributesVoice = dfVoice.drop('label', 1)
dfLabelVoice = dfVoice['label']

Una vez cargados los datasets, debemos dividirlos en dos conjuntos de Train y Test para poder realizar nuestros experimentos.

In [5]:
# Divide into train/test split for our experiments
from sklearn.model_selection import train_test_split
train_attsVoice, test_attsVoice, train_labelVoice, test_labelVoice = train_test_split( 
    dfAttributesVoice,
    dfLabelVoice,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelVoice)

# 1. Selección y evaluación de modelos

En esta sección realizaremos experimentos utilizando el algoritmo GridSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN para el dataset Voice.

In [6]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

Los pasos a seguir en este apartado son:
* Crear un Pipeline con dos pasos:
    - Un objeto de la clase Imputer que se encarga de realizar el tratamiento de valores perdidos. En nuestro caso, imputaremos los valores NaN de los datasets con la media.
    - Un clasificador como estimador (árbol de decisión o KNN en nuestro caso).
* Crear un objeto de la clase GridSearchCV, al que le pasaremos:
    - el estimador que usamos en el Pipeline, 
    - los hiperparámetros del clasificador que queremos tener en cuenta a la hora de buscar la configuración óptima.
    - También debemos indicar qué tipo de métrica usaremos para valorar las configuraciones,
    - cuántos folds se deben realizar en el proceso de validación cruzada que realiza el GridSearchCV. Si en este campo especificamos un objeto de la clase StratifiedKFold con 10 folds y nuestra semilla, podrá realizar un proceso de validación cruzada estratificada.
    - iid = False, para que la evaluación de los resultados se haga sobre una media aritmética, y no sobre una media ponderada.
* Entrenar y validar el clasificador con los datos de nuestros datasets, y analizar los resultados obtenidos (accuracy, precision y recall).

## 1.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

Comenzamos con el clasificador DecisionTree.

In [7]:
from sklearn.preprocessing import Imputer
# Cargamos el arbol de decision
from sklearn import tree

from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

### 1.1.1 Pipeline

A continuación creamos un Pipeline con dos pasos: el primero es imputar los valores perdidos siguiendo la estrategia de rellenar con la media, y el segundo paso es un clasificador DecisionTree, con su semilla para poder hacer reproducibles nuestros experimentos.

In [8]:
from sklearn.pipeline import Pipeline

In [9]:
# We define the pipeline as a set of tuples
estimatorTree = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("Tree", tree.DecisionTreeClassifier(random_state=seed))])

### 1.1.2. GridSearch

Aquí definimos nuestro GridSearch con los parámetros elegidos. Como estimador debemos pasarle el que usamos en nuestro Pipeline.

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [10]:
clfTree = GridSearchCV(
    estimator = estimatorTree,
    param_grid =
        {'Tree__criterion': ["entropy","gini"],'Tree__max_depth': [3, 5, 10, None], 'Tree__min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

Si asignamos el valor 'None' al parámetro 'max_depth', el clasificador intentará llegar a la máxima profundidad del árbol por defecto.

Después de pasar el GridSearch, ejecutamos con el dataset para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro GridSearch.

In [11]:
# We can fit and use the pipeline as usual
clsTreeVoice = clfTree.fit(train_attsVoice, train_labelVoice)
predictionTreeVoice = clfTree.predict(test_attsVoice)
print('Accuracy Voice Tree:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoice)

Accuracy Voice Tree:


0.9605678233438486

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el GridSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 5, y un número mínimo de ejemplos por hoja igual a 10.

In [13]:
print('Mejor configuración de parámetros (Voice Tree):')
clsTreeVoice.best_params_

Mejor configuración de parámetros (Voice Tree):


{'Tree__criterion': 'entropy',
 'Tree__max_depth': 5,
 'Tree__min_samples_leaf': 10}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [14]:
print('Confusion matrix Voice Tree:')
metrics.confusion_matrix(test_labelVoice, predictionTreeVoice)

Confusion matrix Voice Tree:


array([[307,  10],
       [ 15, 302]])

In [15]:
# Recall
print('Recall Voice Tree:')
metrics.recall_score(test_labelVoice, predictionTreeVoice, pos_label="female")

Recall Voice Tree:


0.96845425867507884

In [16]:
# Precision
print('Precision Voice Tree:')
metrics.precision_score(test_labelVoice, predictionTreeVoice, pos_label="female")

Precision Voice Tree:


0.95341614906832295

## 1.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador para nuestro Pipeline.

In [17]:
from sklearn import neighbors

In [18]:
import sklearn.metrics as metrics

In [19]:
from sklearn.model_selection import GridSearchCV

### 1.2.2. Pipeline

Creamos el Pipeline con el imputer como primer paso, y como segundo paso, el clasificador KNN.

In [24]:
# We define the pipeline as a set of tuples
estimatorKNN = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("KNN", neighbors.KNeighborsClassifier())])

### 1.2.1. GridSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el GridSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el GridSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [25]:
clfKNN = GridSearchCV(
    estimator = estimatorKNN,
    param_grid = 
        { 'KNN__n_neighbors' : [1,2,3,4,5], 'KNN__weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset.

In [26]:
# We can fit and use the pipeline as usual
clsKNNVoice = clfKNN.fit(train_attsVoice, train_labelVoice)
predictionKNNVoice = clfKNN.predict(test_attsVoice)
print('Accuracy Voice KNN:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoice)

Accuracy Voice KNN:


0.71135646687697163

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el GridSearch. En este caso es utilizar 4 vecinos, usando la métrica inversa para medir las distancias.

In [27]:
print('Mejor configuración de parámetros (Voice KNN):')
clsKNNVoice.best_params_

Mejor configuración de parámetros (Voice KNN):


{'KNN__n_neighbors': 4, 'KNN__weights': 'distance'}

A continuación mostramos la matriz de confusión, precision y recall obtenidos.

In [30]:
print('Confusion matrix Voice KNN:')
metrics.confusion_matrix(test_labelVoice, predictionKNNVoice)

Confusion matrix Voice KNN:


array([[215, 102],
       [ 81, 236]])

In [31]:
# Recall
print('Recall Voice KNN:')
metrics.recall_score(test_labelVoice, predictionKNNVoice, pos_label="female")

Recall Voice KNN:


0.67823343848580442

In [32]:
# Precision
print('Precision Voice KNN:')
metrics.precision_score(test_labelVoice, predictionKNNVoice, pos_label="female")

Precision Voice KNN:


0.72635135135135132

## 1.3. Comparativa

En este apartado intentaremos analizar cuál de los clasificadores ha obtenido mejores resultados en el dataset estudiado.

Los resultados de accuracy para el árbol de decisión y el KNN con k=4, son: 
* Accuracy Voice Tree: 0.9605678233438486
* Accuracy Voice 4-NN: 0.71135646687697163

En cuanto a los resultados de recall tenemos:
* Recall Voice Tree: 0.96845425867507884
* Recall Voice 4-NN: 0.67823343848580442

Y los de precision:
* Precision Voice Tree: 0.95341614906832295
* Precision Voice 4-NN: 0.72635135135135132

Por tanto, podemos concluir que el mejor modelo es el obtenido con el árbol de decisión.

# 2. Implementación de GridSearch manualmente

En este apartado realizamos una implementación manual de un GridSearh básico, con su constructor, y las funciones fit y predict, que son las que utilizamos en esta práctica.

Nuestra clase GridS tiene como atributos:
* estim: estimador o clasificador a evaluar,
* crossval: cómo realizar el proceso de validación cruzada durante la evaluación,
* score: tipo de métrica a utilizar para la evaluación de la configuración óptima,
* keys: nombre de los parámetros del clasificador a tener en cuenta en la evaluación,
* v: lista con todas las posibles combinaciones de parámetros del clasificador,
* bestScore: mejor puntuación obtenida con la configuración de parámetros del clasificador,
* bestParams: mejor configuración de parámetros del clasificador.

La función fit se encarga de realizar la validación cruzada para cada una de las diferentes combinaciones de parámetros contenidas en self.v, obteniendo así aquella configuración que ofrezca una mayor puntuación. Una vez obtenida, entrena el clasificador con el conjunto de datos que se le pasa como parámetro a la función fit.

La función predict simplemente hace la validación del clasificador con el conjunto que reciba como parámetro.

In [33]:
from itertools import product
from sklearn.model_selection import cross_val_score,StratifiedKFold

class GridS(object):
    def __init__(self,estim, paramG, score, crossval):
        items = paramG.items()
        self.keys, values = zip(*items)
        self.v = list(product(*values))
        self.estim=estim
        self.crossval=crossval
        self.score=score
        self.bestParams = []
            
    def fit(self,atts,label):
        self.bestScore=0
        for param in self.v:
            params = dict(zip(self.keys,param))
            self.estim.set_params(**params)
            scores = cross_val_score(estimator=self.estim, 
                                     X=atts, y=label, 
                                     scoring=self.score, 
                                     cv=self.crossval)
            if self.bestScore < scores.mean():
                self.bestScore=scores.mean()
                self.bestParams=params
        self.estim.set_params(**self.bestParams)
        self.estim.fit(atts,label)
        return self
    
    def predict(self, atts):
        return self.estim.predict(atts)

## 2.1. KNN con GridSearch manual

Creamos un Pipeline que tiene como primer paso el Imputer de los datos, y como segundo paso el clasificador KNN.

In [34]:
estimatorGSKNN = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("myKNN", neighbors.KNeighborsClassifier())])

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos nuestro pipeline, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al número de vecinos (de 1 a 5 vecinos), y el tipo de métrica a utilizar para valorar la distancia (uniforme o inversa de la distancia). El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla. 

In [35]:
gsKNN = GridS(
    estim = estimatorGSKNN,
    paramG = 
        { 'myKNN__n_neighbors' : [1,2,3,4,5], 'myKNN__weights': ['uniform','distance'] },
    score = 'accuracy',
    crossval = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

A continuación utilizamos nuestro GridS en el dataset y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [36]:
clsKNNVoiceGS = gsKNN.fit(train_attsVoice, train_labelVoice)
predictionKNNVoiceGS = gsKNN.predict(test_attsVoice)
print('Accuracy Voice KNN + GridS:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoiceGS)

Accuracy Voice KNN + GridS:


0.71135646687697163

In [37]:
print('Mejor configuración de parámetros (Voice KNN + GridS):')
clsKNNVoiceGS.bestParams

Mejor configuración de parámetros (Voice KNN + GridS):


{'myKNN__n_neighbors': 4, 'myKNN__weights': 'distance'}

In [40]:
# Precision
print('Precision Voice KNN + GridS:')
metrics.precision_score(test_labelVoice, predictionKNNVoiceGS, pos_label="female")

Precision Voice KNN + GridS:


0.72635135135135132

In [41]:
# Recall
print('Recall Voice KNN + GridS:')
metrics.recall_score(test_labelVoice, predictionKNNVoiceGS, pos_label="female")

Recall Voice KNN + GridS:


0.67823343848580442

## 2.2. Árbol de decisión con GridSearch manual

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos un clasificador DecisionTree, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al criterio de evaluación (entropía o gini), la máxima profundidad del árbol, y el número mínimo de ejemplos por hoja. El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla.

In [42]:
estimatorGSTree = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("myTree", tree.DecisionTreeClassifier(random_state=seed),)])

In [43]:
gsTree=GridS(
    estim = estimatorGSTree,
    paramG=
        {'myTree__criterion': ["entropy","gini"],'myTree__max_depth': [3, 5, 10, None], 'myTree__min_samples_leaf': [3,5,10]},
    score='accuracy',
    crossval=StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

A continuación utilizamos nuestro GridS en el dataset y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [44]:
# We can fit and use the pipeline as usual
clsTreeVoiceGS = gsTree.fit(train_attsVoice, train_labelVoice)
predictionTreeVoiceGS = gsTree.predict(test_attsVoice)
print('Accuracy Voice Tree + GridS:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoiceGS)

Accuracy Voice Tree + GridS:


0.9605678233438486

In [45]:
print('Mejor configuración de parámetros (Voice Tree + GridS):')
clsTreeVoiceGS.bestParams

Mejor configuración de parámetros (Voice Tree + GridS):


{'myTree__criterion': 'entropy',
 'myTree__max_depth': 5,
 'myTree__min_samples_leaf': 10}

In [46]:
# Precision
print('Precision Voice Tree + GridS:')
metrics.precision_score(test_labelVoice, predictionTreeVoiceGS, pos_label="female")

Precision Voice Tree + GridS:


0.95341614906832295

In [47]:
# Recall
print('Recall Voice Tree + GridS:')
metrics.recall_score(test_labelVoice, predictionTreeVoiceGS, pos_label="female")

Recall Voice Tree + GridS:


0.96845425867507884

# 4. Extras: Algoritmo RandomizedSearchCV

In [50]:
from sklearn.model_selection import RandomizedSearchCV

En esta sección realizaremos experimentos utilizando el algoritmo RandomizedSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN
. Seguiremos el mismo proceso que hicimos con GridSearch.

## 4.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

### 4.1.1. Pipeline

Primero creamos un Pipeline con dos pasos: el primero es imputar los valores perdidos siguiendo la estrategia de rellenar con la media, y el segundo paso es un árbol.

In [51]:
# We define the pipeline as a set of tuples
estimatorTreeRS = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("RSTree", tree.DecisionTreeClassifier(random_state=seed))])

### 4.1.2. RandomizedSearch

Aquí definimos nuestro RandomizedSearch con los parámetros elegidos, usando nuestro Pipeline.

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [52]:
clfTreeRS = RandomizedSearchCV(
    estimator = estimatorTreeRS,
    param_distributions =
        {'RSTree__criterion': ["entropy","gini"],'RSTree__max_depth': [3, 5, 10, None], 'RSTree__min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

Después de crear el Pipeline, lo ejecutamos con el dataset para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro RandomizedSearch.

In [53]:
# We can fit and use the pipeline as usual
clsTreeVoiceRS = clfTreeRS.fit(train_attsVoice, train_labelVoice)
predictionTreeVoiceRS = clfTreeRS.predict(test_attsVoice)
print('Accuracy Voice Tree with RS:')
metrics.accuracy_score(test_labelVoice, predictionTreeVoice)

Accuracy Voice Tree with RS:


0.9605678233438486

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el RandomizedSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 5, y un número mínimo de ejemplos por hoja igual a 5.

In [55]:
print('Mejor configuración de parámetros (Voice Tree) con RS:')
clsTreeVoiceRS.best_params_

Mejor configuración de parámetros (Voice Tree) con RS:


{'RSTree__criterion': 'entropy',
 'RSTree__max_depth': 5,
 'RSTree__min_samples_leaf': 5}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [56]:
print('Confusion matrix Voice Tree with RS:')
metrics.confusion_matrix(test_labelVoice, predictionTreeVoiceRS)

Confusion matrix Voice Tree with RS:


array([[305,  12],
       [ 15, 302]])

In [58]:
# Recall
print('Recall Voice Tree with RS:')
metrics.recall_score(test_labelVoice, predictionTreeVoiceRS, pos_label="female")

Recall Voice Tree with RS:


0.96214511041009465

In [59]:
# Precision
print('Precision Voice Tree with RS:')
metrics.precision_score(test_labelVoice, predictionTreeVoiceRS, pos_label="female")

Precision Voice Tree with RS:


0.953125

## 4.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador para nuestro Pipeline.

### 4.2.1. Pipeline

Creamos el Pipeline con el imputer como primer paso, y como segundo paso, el clasificador KNN.

In [60]:
# We define the pipeline as a set of tuples
estimatorKNNRS = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("RSKNN", neighbors.KNeighborsClassifier())])

### 4.2.2. RandomizedSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el RandomizedSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el RandomizedSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [61]:
clfKNNRS = RandomizedSearchCV(
    estimator = estimatorKNNRS,
    param_distributions = 
        { 'RSKNN__n_neighbors' : [1,2,3,4,5], 'RSKNN__weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset.

In [62]:
# We can fit and use the pipeline as usual
clsKNNVoiceRS = clfKNNRS.fit(train_attsVoice, train_labelVoice)
predictionKNNVoiceRS = clfKNNRS.predict(test_attsVoice)
print('Accuracy Voice KNN with RS:')
metrics.accuracy_score(test_labelVoice, predictionKNNVoiceRS)

Accuracy Voice KNN with RS:


0.71135646687697163

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el RandomizedSearch. En este caso es utilizar 4 vecinos, usando la métrica inversa para medir las distancias.

In [63]:
print('Mejor configuración de parámetros (Voice KNN) con RS:')
clsKNNVoiceRS.best_params_

Mejor configuración de parámetros (Voice KNN) con RS:


{'RSKNN__n_neighbors': 4, 'RSKNN__weights': 'distance'}

A continuación mostramos la matriz de confusión, precision y recall obtenidos.

In [64]:
print('Confusion matrix Voice KNN with RS:')
metrics.confusion_matrix(test_labelVoice, predictionKNNVoiceRS)

Confusion matrix Voice KNN with RS:


array([[215, 102],
       [ 81, 236]])

In [65]:
# Recall
print('Recall Voice KNN with RS:')
metrics.recall_score(test_labelVoice, predictionKNNVoiceRS, pos_label="female")

Recall Voice KNN with RS:


0.67823343848580442

In [66]:
# Precision
print('Precision Voice KNN with RS:')
metrics.precision_score(test_labelVoice, predictionKNNVoiceRS, pos_label="female")

Precision Voice KNN with RS:


0.72635135135135132

Como podemos observar, los resultados usando RandomizedSearch y GridSearch son muy similares.